This notebook demonstrates an example of running a single dataset through the lab's full analysis pipeline for NMF-based models.

The main steps involved are:
##### 1. Format Data \[MATLAB\]
##### 2. Preprocessing \[MATLAB\]
##### 3. Prepare data in Python \[Python\]
##### 4. Train and evaluate NMF model \[Python\]
##### 5. Save results \[Python\]
##### 6. Interpret results \[Python & MATLAB\]

# Step 1: Format Data

   The recorded data must be saved into a folder with the following structure:  
   
   The folder must have a subfolder named *Data* that contains a *.mat* file for each recording session.
   Those files should have the following naming scheme: _**mouse-name**\_**date**\_**experiment-identifier**\_LFP.mat_ (example: *Mouse798_021519_CUS_LFP.mat*).
   Each file in data should contain variables containing vectors of recorded data corresponding to each of the channels recorded in that session.  
   
   The same folder containing data must have another subfolder named *CHANS* that has *.mat* files corresponding to each of the files in *Data*.
   Each file in *CHANS* should have the follwing naming scheme:  _**mouse-name**\_**date**\_**experiment-identifier**\_CHANS.mat_ (example: *Mouse798_021519_CUS_CHANS.mat*).
   Each of these files should contain 2 variables, **CHANNAMES** and **CHANACTIVE**.
   **CHANNAMES** should be an array of strings giving the names of each of the channels in the corresponding LFP file.
   **CHANACTIVE** should be a vector of 1s or 0s.
   If **CHANACTIVE**(k) = 0, it indicates that the channel given by **CHANNAMES**(k) should not be included in preprocessing; conversely, **CHANACTIVE**(k) = 1 means that **CHANNAMES**(k) should be included in preprocessing.
   For more info on converting recorded data from the *.ns2* format to the format described here, ask Steve Mague. 
   
   You must also create an excel spreadsheet with two columns containing channel information.
   The first column should be a master list of all the channel names that will appear in the dataset (e.g. **NAc_Shell_02**).
   The second column should give the abbreviated name of the area that each channel corresponds to; for instance you might list **NAc_Shell** as the area for the channel **NAc_Shell_02**.
   You can use the area labels to combine data from different hemispheres or from multiple nearby regions.
   For example, the channels **NAc_Shell_02** and **NAc_Core_01** might both have **NAc** listed as their corresponding area if you wish to combine data from Nucleus Accumbens core and shell.
   
   If the optional *useIntervals* parameter is set to **true**, then the project folder must also contain a subfolder named *INT_TIME*.
   Setting *useIntervals* to **true** will allow you to select only specific intervals of the recording to preprocess and save.
   That folder should contain *.mat* files corresponding to each file in the *Data* folder with the following naming scheme: _**mouse-name**\_**date**\_**experiment-identifier**\_TIME.mat_ (example: *Mouse798_021519_CUS_TIME.mat*).
   Each of those files will contain a single vector variable named **INT_TIME** that should have *2N* entries, where *N* is the total number of intervals you wish to use.
   The odd entries (1,3,5,...) contain the starting time (in seconds) of each interval, and the even entries (2,4,6...) contain the duration of each interval in seconds.

   To format recorded data into windows, run *formatWindows* in MATLAB. *formatWindows* takes one parameter, which names the file that you will save the formatted data to.                                                                                 

Example:                                                                                                                       

```matlab
formatWindows('myTestFile.mat')
``` 

# Step 2a: Preprocessing

 To preprocess formatted data, run *preprocessData* in MATLAB. *preprocessData* has one required input parameter, which should be the same filename given to *formatWindows*.
A second optional parameter can be used to change preprocessing options.

run in MATLAB: ```preprocessData(myTestFile.mat, dataOpts)```

 
 Replace 'myTestFile.mat' with the name/location of the file containing your data. *dataOpts* is a structure that you can use to optionally change some of the settings for preprocessing; you can use it to change the following settings:
* subSampFact: subsampling factor
* normWindows: String indicating of how to normalize data. If 'window', each window is normalized seperately. If 'whole', dataset is normalized as a whole, and individual windows are still set to have zero mean. Set to 'mouse' to normalize channels seperately for each mouse, or 'day' to normalize within day for each channel for each mouse. Can be set to 'none' if no normalization is desired. Default is 'day'.
* transform: whether or not to take and save the Fourier transform of the data

Additionally, if you plan on using the data you are preprocessing to 'backproject' into another model that was trained using other data, you will want to preprocess this data with some of the preprocessing parameters used on the training set. To do that, load the *dataOpts* variable from the file where your preprocessed training set is saved and use that as the *dataOpts* parameter for *preprocessData*.

# Step 2b. Calculate frequency-based features
**In MATLAB:**

Run the saveFeatures function to generate features such as power, coherence, or Granger causality.

```matlab
opts.featureList = {'power', 'coherence', 'granger'}; 
opts.mvgcFolder = '~/lpne-data-analysis/mvgc/'
 
saveFeatures(‘myTestFile.mat’, opts);
```
Again, replace 'myTestFile.mat' with the name/location of the file containing your data. opts is a structure that you can use to optionally change some of the settings for feature generate; you can use it to change the following settings:
* featureList: cell array of features to calculate. Options are 'power', 'coherence', and/or 'granger'
* mvgcFolder: (Only needed if you want to calculate granger features) String giving path to MVGC toolbox. There is a copy of this toolbox inside the lpne-data-analysis repository, so you should be able to use that location
* parCores: integer indicating number of cores to use for  parallel computing. If 0 (default), all tasks executed in serial.
* windowOpts:  a binary vector with length=number of windows, determining which windows to analyze for features.

# Step 3. Prepare Data in Python

**NOTE:** To run the Python components of this notebook, you will need a matlab (.mat) file containing variables 'power', 'coherence', and/or 'granger', and a JSON file containing 'labels'.
You will also need the lpne-data-analysis git repository as well as the NMF_tf repository.

Demo data to run this can be found at LabCommon/AnalysisDemoData.

### Edit paths to pipeline, NMF_tf, and datafile

In [ ]:
from pathlib import Path
my_home = str(Path.home())

# Edit pipeline path here
pipepath = my_home + '/lpne-data-analysis'
# Edit path to NMF_tf folder here
nmfpath = my_home + '/NMF_tf/'
# Enter path to '.mat' data file here
datapath = '/work/nmg14/TST/TeST.mat'

# set name for nmf models to be saved with
name = 'testModel'

#### import required modules

In [ ]:
import sys
import os
sys.path.append(pipepath)
sys.path.append(my_home)
sys.path.append(nmfpath)

import data_tools
import validation_tools
import numpy as np
import norm_base, norm_supervised
import pickle
%matplotlib notebook
import matplotlib.pyplot as plt

### load and extract the data
*load_features* should be a list of strings describing the features you want to extract from the file given by *datapath*. 
Typically the options you would choose are 'power', 'coherence', 'granger', and/or 'instant' (instantaneous causality). 
The features are returned in the order that you list them. 

The last return value is a dictionary of labels that includes the following keys:            
* 'windows': Dictionary containing information associated with each individual time-window. Some options for keys included in this dictionary inclue 'task' (i.e. task being done by mouse) and 'mouse' (i.e. name of mouse).
* 'powerFeatures': list of string labels describing the features represented in power.
* 'cohFeatures': list of string labels describing the features represented in coherence
* 'gcFeatures': list of string labels describing the features represented in granger
* 'instFeatures': list of string labels describing the features represented in instant

In [ ]:
load_features = ['power', 'granger']

power, granger, labels = \
data_tools.load_data(datapath, feature_list=load_features, f_bounds=(1,55))

### generate one-hot vector for output
The *target* should be the key in labels\['windows'\] that you would like to extract

In [ ]:
target = 'task'
y = data_tools.get_labels(labels, target)

### (optional) group mice together to balance some desired variable
You may want your training/test sets to maintain balanced numbers of some variable (e.g. genotype). To do this the second input parameter should be some key in labels\['windows'\]. If you just want to group all of the windows for individal mice together, run the commented out line below instead.

In [ ]:
groups = validation_tools.get_balanced_groupings(labels, 'genotype')

#groups = data_tools.get_labels(labels, 'mouse')

# Step 4: Train and evaluate NMF model

### choose model and training options
To use cross-validation to choose to between multiple options for any parameter, set that parameter to a list of all the options you would like to consider. Comment out the line associated with any option to use the default value.

In [ ]:
model_opts = {'train func':norm_base.train_model,
              'eval func':norm_base.evaluate_model}

params=dict()
params['dir name'] = name + 'Dir'   
# directory to which you would like to save model checkpoints

params['name'] = name   
# name associated with your model

params['n components'] = 5   
# number of factors in model

params['learning rate'] = 1e-4  
# learning rate

params['superstrength'] = 3   
# strength of supervision

params['n iter'] = 150000   
# number of training iterations

params['NMF variant'] = norm_supervised.sNMF   
# model class that you would like to use within the NMF_tf repository

params['feature_weights'] = [[1,1]]   
# relative weighting of features included in 'feature_list'
# Must be a list of lists

gen = data_tools.get_labels(labels, 'genotype')
params['samp weights'] =  np.ones_like(gen) + 2*np.asarray(gen) 
# use 'samp weights' if you want to weight windows differently when training the model. This
# example weights windows from a specific genotyp 3x.

params['repeats'] = 5
# number of times to repeat each combination of parameters
# the best training session will be take from the repeats

feature_list = [power, granger]

### **Option A:** Use cross-validation to learn a model
Cross-validation is useful when you want to test multiple values for one or more parameters. We use grid search cross-validation, which will train a model for each combination of parameter values you want to test on multiple different subsets of the data. The run_cv method will do this and then train a final model using the combination of parameters that performs the best over all of the data subsets (more specifically on the data that was held out from each subset).

Change *folds* to the number of cross-validation folds you would like to use. 

Set *metric* to 'accuracy', 'auc' or 'precision' if you want to use the area under the ROC curve or precision-recall curve, respectively, to evaluate performance. Generally 'auc' or 'precision' are preferred over 'accuracy', but 'accuracy' is arguably more interpretable in multi-class situations. 'auc' is most appropriate when you have approximately the same number of observations for each class in *y* (e.g. half of the observations are class 'A' and half are class 'B') 'precision' is more appropriate when your classes are not balanced (when using 'precision' the class with fewer observations should be the positive class). A good rule of thumb is to use 'precision' when the ratio between classes is greater than 5:1.

In [ ]:
%%capture cv_output
# this line saves the training output without printing it
# To print it later run 'cv_output.show()'

cv_results = validation_tools.run_cv(feature_list, y, groups, folds=3, metric='auc', 
                                parameters=params, modelOpts=model_opts)

In [ ]:
cv_output.show()

### **Option B:** Use nested cross-validation to estimate the performance of a *class* of models
Nested cross-validation is useful when you want an unbiased estimate of how well a certain class of models performs on your data. In nested cross-validation you perform cross-validation on multiple subset of your data and get a seperate final model for each subset. The performance of each model is then evaluated on the portion of the data not included in the subset it was trained on, giving a set of unbiased performance values for each subset of data. You can use these performance values to estimate the mean and variance of the type of model you are using on your dataset. **Note:** This is different than traditional cross-validation because you do not finish with a single final model for interpretation.

In [ ]:
%%capture ncv_output

ncv_results = validation_tools.run_nested_cv(feature_list, y, groups, folds=5, metric='auc', 
                                parameters=params, modelOpts=model_opts)

### Backproject  holdout dataset into model
Repeat data loading and preparation for holdout set as you did for the training set. See comments below for details

In [ ]:
# Edit path to holdout data
holdout_path = '/home/data_store1/TST/TeST.mat'

# set this to have the same number of outputs as for the training set
power2, granger2, labels2 = \
data_tools.load_data(holdout_path, feature_list=load_features)
y2 = data_tools.get_labels(labels2, target)

feat_weights = cv_results['parameters']['feature_weights']

# Choose the same set of features as training to project into the model
feature_list2 = [power2, granger2]
X2 = data_tools.get_X(feat_weights, feature_list2)
holdout_perf, holdout_data = norm_base.evaluate_model(cv_results['model'], data=(X2, y2), 
                                                      metric='accuracy')

### print performance statistics
Prints the performance metric (e.g. AUC) on each of the test set splits, along with mean and standard deviation

In [ ]:
perf = ncv_results['performance']
print('Nested Cross-validation performance: \n', perf)
print('mean: %.2f -  SD: %.2f' % (np.mean(perf), np.std(perf)))

print('\n Holdout Performance:\n', holdout_perf)

# 5. Save results

In [ ]:
nmf_model = cv_results['model'][0]
factors = nmf_model.components_

savedata = cv_results.copy()
savedata['factors'] = factors
savedata['classifier'] = cv_results['model'][1]
savedata.pop('model',None)

pickle.dump(savedata, open('cvResults.p','wb'))

# 6. Interpret results

In [ ]:
print(labels['area'])

### Plot features included in factors

Regions in the feature set are listed above. To plot power, set *plot_feature* to 'p *region*', where *region* is one of the regions listed above. To plot Granger causality set *plot_feature* to '*region1*->*region2*'. To plot coherence set *plot_feature* to '*region1*-*region2*' (for coherence there is only one correct ordering for a given pair of regions).

In [ ]:
plot_feature = 'IL_Cx-Acb_Sh'
factor_no = 0 # supervised factors come first
# assumes that feature_list used to train model contains 
# power and granger features, in that order
feature_labels = np.hstack((labels['powerFeatures'], labels['gcFeatures']))

validation_tools.plot_factors(factors, plot_feature, factor_no, feature_labels)

### Plot factor scores over time (for holdout set)

In [ ]:
factor_no = 2

plt.plot(holdout_data['scores'][:,factor_no])
plt.show()

In [ ]:
# re-import modules if they've been edited

import importlib
importlib.reload(norm_base)